# Multi-agent System with AgentWorkflow


Our system will have three agents:

*  A `ResearchAgent` that searches the web for information on the given topic.
*  A `WriteAgent` that will write the report using the information found by the `ResearchAgent`.
*  A `ReviewAgent` that reviews the report and provides feedback.

 we will use a few tools to help with the research and writing processes.

1. A `web_search` tool to search the web for information on the given topic.
2. A `record_notes` tool which will save research found on the web to the state so that the other tools can use it.
3. A `write_report` tool to write the report using the information found by the `ResearchAgent`.
4. A `review_report` tool to review the report and provide feedback.

Utilizing the `Context` class, we can pass state between agents, and each agent will have access to the current state of the system.



In [ ]:
!pip install llama-index -q -q

In [ ]:
!pip install tavily-python -q -q

In [ ]:
from tavily import AsyncTavilyClient
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.openai import OpenAI
import os
from openai import OpenAI as OpenAIClient

raw_client = OpenAIClient()

API_KEY   = raw_client.api_key
API_BASE  = raw_client.base_url

tavily_api_key = os.environ["TAVILY_API_KEY"]

llm = OpenAI(model="gpt-4o-mini", max_tokens=64, api_key=API_KEY, api_base=API_BASE)

from tavily import AsyncTavilyClient
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.openai import OpenAI
import os
from openai import OpenAI as OpenAIClient

raw_client = OpenAIClient()

API_KEY   = raw_client.api_key
API_BASE  = raw_client.base_url

tavily_api_key = os.environ["TAVILY_API_KEY"]

llm = OpenAI(model="gpt-4o-mini", max_tokens=64, api_key=API_KEY, api_base=API_BASE)

result_fallback = "Internet history timeline: ARPANET to the World Wide Web In internet history, credit for the initial concept that developed into the World Wide Web is typically given to Leonard Kleinrock. The precursor to the internet was jumpstarted in the early days of the history of computers , in 1969 with the U.S. Defense Department's Advanced Research Projects Agency Network (ARPANET), according to the journal American Scientist. 1989: World.std.com becomes the first commercial provider of dial-up access to the internet. 1989: World.std.com becomes the first commercial provider of dial-up access to the internet. 1990: Tim Berners-Lee develops the foundational technologies of the World Wide Web, including HTML, HTTP, and URLs, while working at CERN. 1991: The World Wide Web is launched to the public, making internet browsing accessible to non-technical users. 1993: Mosaic, the first widely-used graphical web browser, is released, helping to popularize the Web. 1995: Commercialization of the internet begins, with companies like Amazon and eBay launching online."

async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=tavily_api_key)
    result = await client.search(query, max_results=1)
    if result.get("answer"):
        return result["answer"]
    else:
        return result_fallback

In [ ]:
from llama_index.core.workflow import Context

async def record_notes(ctx: Context, notes: str, notes_title: str = "Untitled Notes") -> str:
    """Useful for recording notes on a given topic."""
    current_state = await ctx.get("state")
    if "research_notes" not in current_state:
        current_state["research_notes"] = {}
    current_state["research_notes"][notes_title] = notes
    await ctx.set("state", current_state)
    return "Notes recorded."


async def write_report(ctx: Context, report_content: str) -> str:
    """Useful for writing a report on a given topic."""
    current_state = await ctx.get("state")
    current_state["report_content"] = report_content
    await ctx.set("state", current_state)
    return "Report written."


async def review_report(ctx: Context, review: str) -> str:
    """Useful for reviewing a report and providing feedback."""
    current_state = await ctx.get("state")
    current_state["review"] = review
    await ctx.set("state", current_state)
    return "Report reviewed."

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent

research_agent = FunctionAgent(
    name="ResearchAgent",
    description="Useful for searching the web for information\
    on a given topic and recording notes on the topic.",
    system_prompt=(
        "You are the ResearchAgent that can search the web for\
        information on a given topic and record notes on the topic. "
        "Once notes are recorded, you should hand off \
        control to the WriteAgent to write a report on the topic."
    ),
    llm=llm,
    tools=[search_web, record_notes],
    can_handoff_to=["WriteAgent"],
)

write_agent = FunctionAgent(
    name="WriteAgent",
    description="Useful for writing a report on a given topic.",
    system_prompt=(
        "You are the WriteAgent that can write a report on a given topic. "
        "Your report should be in a markdown format. The content should be grounded in the research notes. "
        "Once the report is written, you should get feedback from the ReviewAgent."
    ),
    llm=llm,
    tools=[write_report],
    can_handoff_to=["ReviewAgent", "ResearchAgent"],
)

review_agent = FunctionAgent(
    name="ReviewAgent",
    description="Useful for reviewing a report and providing feedback.",
    system_prompt=(
        "You are the ReviewAgent that can review a report and provide feedback. "
        "Your feedback should either approve the current report or request changes for the WriteAgent to implement."
    ),
    llm=llm,
    tools=[review_report]
)

##  Running the Workflow



In [ ]:
from llama_index.core.agent.workflow import AgentWorkflow

agent_workflow = AgentWorkflow(
    agents=[research_agent, write_agent, review_agent],
    root_agent=research_agent.name,
    initial_state={
        "research_notes": {},
        "report_content": "Not written yet.",
        "review": "Review required.",
    },
)

an idea of what is happening under the hood.

In [ ]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = agent_workflow.run(
    user_msg="""Write me a report on the history of the web.
      Briefly describe the history of the world wide web, including
      the development of the internet and the development of the web,
      including 21st century developments""",
    max_handoffs=1
)

current_agent = None
current_tool_calls = ""
async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")


🤖 Agent: ResearchAgent

🛠️  Planning to use tools: ['search_web']
🔨 Calling Tool: search_web
  With arguments: {'query': 'history of the world wide web and internet development 21st century'}
🔧 Tool Result (search_web):
  Arguments: {'query': 'history of the world wide web and internet development 21st century'}
  Output: Internet history timeline: ARPANET to the World Wide Web In internet history, credit for the initial concept that developed into the World Wide Web is typically given to Leonard Kleinrock. The precursor to the internet was jumpstarted in the early days of the history of computers , in 1969 with the U.S. Defense Department's Advanced Research Projects Agency Network (ARPANET), according to the journal American Scientist. 1989: World.std.com becomes the first commercial provider of dial-up access to the internet. 1989: World.std.com becomes the first commercial provider of dial-up access to the internet. 1990: Tim Berners-Lee develops the foundational technologies of t

## ✨ Congratulations again!

We've successfully put together a multi-agent system using `AgentWorkflow`. `AgentWorkflow` is designed to be a simple jumping-off point for putting together a team of agents to solve a problem. But LlamaIndex can do even more!